# DSCI445 Term Project Paper - Bank Account Fraud Detection
### Jakob Wickham, Nick Brady, Noah Sturgeon

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imblearn as skim
import xgboost as xgb
import kagglehub
import seaborn as sns
import sklearn as sk

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, classification_report

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek

## Introduction



### Motivation

-{Imporanted of fraud dection in finacial service (Provide reference)}
-{Challanges associated with fraud dection: Imbalced datsets , cost associated with false postive / false negatives}
-{Objective of the proeject}:Our goal in this project is to work with various machine learning methods to get the highest recall (how many fraudulent transactions are successfully identified) possible, while locking our false positive rate (how many valid transactions are misidentified) at 5% or lower. The reason for using recall, as opposed to accuracy, is due to the nature of the dataset. There is far less fraud than legitimate transactions, and so aiming for accuracy will inherently create a model that is useless at detecting fraud.

## Dataset overview 

-{Description of the dataset}:
    - Source of the data set 
    - {Key Features} 
    - {Target Variables}

&nbsp;&nbsp;&nbsp;&nbsp;The Bank Account Fraud NeurIPS 2022 datasets (called BAF for short) are a suite of synthetic datasets meant to evaluate machine learning methods. 
BAF was generated with a fraud label, a boolean value called fraud_bool, which is the response on which machine learning methods can be tested on.



## Methodolgy